In [1]:
import asyncio
import websockets
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import threading
import webbrowser
from flask import Flask

# generate sample timestamp


# Generate sample data
data = np.append(
            np.append(
                np.append(
                    np.random.randint(50,61,(100,1)),
                    np.random.randint(0,100,(100,1)), axis=1),
                np.random.randint(-100,0,(100,1)), axis=1),
            np.random.randint(1,3,(100,1)), axis=1)

df = pd.DataFrame(data, columns=["timeStamp", "TxPower","RSSI","RTT"])
df["timeStamp"] = df["timeStamp"].cumsum(axis=0)

In [2]:
df

,timeStamp,TxPower,RSSI,RTT
0,55,37,-70,1
1,107,86,-75,1
2,157,39,-22,1
3,214,54,-29,1
4,264,23,-21,1
...,...,...,...,...
95,5226,30,-58,2
96,5279,43,-90,1
97,5339,28,-8,1
98,5392,35,-21,1


In [35]:
fig = go.Figure()

power_color = '#636EFA'
rssi_color = '#EF553B'
rtt_color = '#00CC96'
grid_color = "rgb(75,75,75)"

fig.update_layout(font_color="rgb(150,150,150)")

fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["TxPower"], name="TxPower", mode="lines", line=dict(color=power_color)))
fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["RSSI"], name="RSSI", mode="lines", line=dict(color=rssi_color), yaxis="y2"))
fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["RTT"], name="RTT", mode="lines", line=dict(color=rtt_color), yaxis="y3"))

fig.update_layout(
                  title=dict(text="Radio Transmission metrics", font_size=24),
                  width=1750,
                  height=800,
                  legend=dict(x=0, y=1.07, orientation='h'),
                  plot_bgcolor='rgb(35,35,35)',
                  paper_bgcolor ='rgb(10,10,10)',
                  xaxis=dict(domain=[0.05,1], linecolor=grid_color, gridcolor=grid_color, mirror=True),
                  yaxis=dict(title=dict(text="TxPower (dBm)", font=dict(color=power_color)), tickfont=dict(color=power_color), range=[0,100], linecolor=grid_color, gridcolor=grid_color),
                  yaxis2=dict(title=dict(text="RSSI (dBm)", font=dict(color=rssi_color)), tickfont=dict(color=rssi_color), anchor="free", overlaying="y", side="left", position=0, range=[-100,0], linecolor=grid_color, gridcolor=grid_color),
                  yaxis3=dict(title=dict(text="RTT (ms)", font=dict(color=rtt_color)), tickfont=dict(color=rtt_color), anchor="x", overlaying="y", side="right", range=[0,5], linecolor=grid_color, gridcolor=grid_color)
                )

fig.show()

In [ ]:

# Initialize Dash app with Flask server
server = Flask(__name__)
app = Dash(__name__, server=server)

app.layout = html.Div([
    dcc.Graph(id='live-graph'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # in milliseconds
        n_intervals=0
    )
])

@app.callback(Output('live-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_plot(n):
    
    df = datagen()
    
    if df.empty:
        return go.Figure()

    tdf = df.iloc[-100:]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=tdf.index, y=tdf['TxPower'], mode='lines', name='TxPower', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=tdf.index, y=tdf['RSSI'], mode='lines', name='RSSI', line=dict(color='blue'), yaxis='y2'))
    fig.add_trace(go.Scatter(x=tdf.index, y=tdf['RTT'], mode='lines', name='RTT', line=dict(color='red'), yaxis='y3'))

    # Update layout for clarity
    material_title = tdf['Material'].iloc[-1] if not tdf.empty else "<unknown>"
    fig.update_layout(
        title=f'Live Data Visualization - {material_title}',
        xaxis=dict(title='Index'),
        yaxis=dict(title='TxPower (dBm)', titlefont=dict(color='green'), tickfont=dict(color='green'), anchor="free", overlaying='y', side='left', position=0, range=[0, 100]),
        yaxis2=dict(title='RSSI (dBm)', titlefont=dict(color='blue'), tickfont=dict(color='blue'), anchor="free", overlaying='y', side='right', position=0, range=[-100, 0]),
        yaxis3=dict(title='RTT (ms)', titlefont=dict(color='red'), tickfont=dict(color='red'), anchor="free", overlaying='y', side='right', position=1, range=[0, 10]),
        legend=dict(x=0, y=1.1, orientation='h')
    )

    return fig

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == '__main__':
    threading.Timer(1, open_browser).start()
    app.run_server(debug=True)